In [229]:
import logging
import os
import sys

import numpy as np
import pandas as pd

sys.path.insert(0, '..')

from app import db

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 50)


<ipython-input-229-f0b900f0fc61>:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [230]:
db_conn = db.get_db_conn()

## Passing Summary

In [232]:
join_cols = ['year', 'season_type', 'game_id', 'team', 'week','opp', 'gsis_id', 'pos', 'player']

"""
teching points:
df = df_pass.merge(df_rush, how='inner', on=join_cols, suffixes=['_pass', '_rush'])
this only rename the shared columns!
e.g. completions only exists in passing so it isnt suffixed in  the merge
"""

def prepare_columns(df, suffix):
    return df.rename(columns={
        col: f"{col}{suffix}" for col in df.columns if col not in join_cols
    })

In [233]:
"""
Columns to keep in the summary:
pass - completions, attempts, yards, air_yards_intended, air_yards_completed, td, int, sacks, sack_yards, fumbles,
fumbles_lost, spikes, epa, epa_pass, epa_int, epa_sack, epa_spike, cpoe, 
"""

query = """SELECT * FROM passing_by_player_by_game"""
df_pass = pd.read_sql(query, db_conn)
df_pass = prepare_columns(df_pass, "_pass")
df_pass.head()

,year,season_type,game_id,team,opp,week,gsis_id,pos,player,completions_pass,attempts_pass,yards_pass,air_yards_intended_pass,air_yards_completed_pass,td_pass,int_pass,sacks_pass,sack_yards_pass,fumbles_pass,fumbles_lost_pass,spikes_pass,epa_pass,epa_pass_pass,epa_int_pass,epa_sack_pass,epa_spike_pass,cpoe_pass,completions_wr_pass,attempts_wr_pass,yards_wr_pass,air_yards_intended_wr_pass,air_yards_completed_wr_pass,td_wr_pass,int_wr_pass,epa_wr_pass,cpoe_wr_pass,completions_te_pass,attempts_te_pass,yards_te_pass,air_yards_intended_te_pass,air_yards_completed_te_pass,td_te_pass,int_te_pass,epa_te_pass,cpoe_te_pass,completions_rb_pass,attempts_rb_pass,yards_rb_pass,air_yards_intended_rb_pass,air_yards_completed_rb_pass,td_rb_pass,int_rb_pass,epa_rb_pass,cpoe_rb_pass,completions_null_pass,attempts_null_pass,yards_null_pass,air_yards_intended_null_pass,air_yards_completed_null_pass,td_null_pass,int_null_pass,epa_null_pass,cpoe_null_pass,completions_other_pass,attempts_other_pass,yards_other_pass,air_yards_intended_other_pass,air_yards_completed_other_pass,td_other_pass,int_other_pass,epa_other_pass,cpoe_other_pass,target_share_wr_pass,target_share_te_pass,target_share_rb_pass,target_share_other_pass,air_yards_intended_share_wr_pass,air_yards_intended_share_te_pass,air_yards_intended_share_rb_pass,air_yards_completed_share_wr_pass,air_yards_completed_share_te_pass,air_yards_completed_share_rb_pass
0,2020,REG,2020_03_KC_BAL,KC,BAL,3,00-0033873,QB,P.Mahomes,31,42,385,346,207,4,0,0,0,0,0,0,31.377849,31.377849,0.0,0.000000,0.000000,9.176690,16,21,220,288,169,2,0,18.443355,393.802585,6,8,87,52,38,0,0,5.017431,9.524000,8,11,76,1,-1,1,0,7.776404,-49.395210,0,1,0,4,0,0,0,-0.764009,0.0,1,1,2,1,1,1,0,0.904668,31.489611,0.500000,0.190476,0.261905,0.02381,0.832370,0.150289,0.002890,0.816425,0.183575,-0.004831
1,2020,REG,2020_06_ATL_MIN,ATL,MIN,6,00-0026143,QB,M.Ryan,30,40,371,270,200,4,0,2,-8,0,0,0,30.441585,33.212461,0.0,-2.770877,0.000000,8.848381,19,25,268,224,170,3,0,23.358781,224.087441,4,4,57,21,21,1,0,8.707816,108.141237,6,7,43,5,6,0,0,3.862365,19.846362,0,3,0,17,0,0,0,-2.419047,0.0,1,1,3,3,3,0,0,-0.297454,19.556969,0.625000,0.100000,0.175000,0.02500,0.829630,0.077778,0.018519,0.850000,0.105000,0.030000
2,2020,REG,2020_08_NYJ_KC,KC,NYJ,8,00-0033873,QB,P.Mahomes,31,42,416,388,218,5,0,0,0,0,0,0,27.690373,27.690373,0.0,0.000000,0.000000,8.220293,14,20,249,294,171,4,0,17.918665,220.967728,10,15,119,105,58,1,0,6.302816,8.920118,7,7,48,-11,-11,0,0,3.468892,115.364480,0,0,0,0,0,0,0,0.000000,0.0,0,0,0,0,0,0,0,0.000000,0.000000,0.476190,0.357143,0.166667,0.00000,0.757732,0.270619,-0.028351,0.784404,0.266055,-0.050459
3,2020,REG,2020_07_TB_LV,TB,LV,7,00-0019596,QB,T.Brady,33,45,369,399,235,4,0,0,0,0,0,2,26.700100,27.056477,0.0,0.000000,-0.356377,13.354659,19,22,238,235,160,3,0,21.308635,473.062178,7,10,82,123,68,1,0,5.583212,138.986549,7,9,49,14,7,0,0,1.715990,-11.089057,0,2,0,27,0,0,0,-1.551360,0.0,0,0,0,0,0,0,0,0.000000,0.000000,0.488889,0.222222,0.200000,0.00000,0.588972,0.308271,0.035088,0.680851,0.289362,0.029787
4,2020,REG,2020_16_TB_DET,TB,DET,16,00-0019596,QB,T.Brady,22,27,348,359,266,4,0,1,-7,0,0,0,26.576751,28.141935,0.0,-1.565183,0.000000,20.157525,14,18,229,286,193,3,0,19.641855,334.387913,4,4,80,66,66,1,0,7.515910,152.923632,4,4,39,7,7,0,0,2.378777,77.099150,0,1,0,0,0,0,0,-1.394607,0.0,0,0,0,0,0,0,0,0.000000,0.000000,0.666667,0.148148,0.148148,0.00000,0.796657,0.183844,0.019499,0.725564,0.248120,0.026316


In [234]:
"""
Columns to keep in the summary:
rush - attempts_totals, yards_totals, td, fumbles, fumbles_lost, epa_total,
attempts_designed, yards_designed, td_designed, epa_designed,
attempts_scrambles, yards_scrambles, td_scrambles,
attempts_kneels, yards_kneels, epa_kneels,
attempts, yards, epa
"""

query = """SELECT * FROM rushing_by_player_by_game"""
df_rush = pd.read_sql(query, db_conn)
df_rush = prepare_columns(df_rush, '_rush')
df_rush.head()

,year,season_type,game_id,team,opp,week,gsis_id,pos,player,attempts_total_rush,yards_total_rush,td_total_rush,fumbles_total_rush,fumbles_lost_total_rush,fumbles_out_of_bounds_total_rush,epa_total_rush,attempts_designed_rush,yards_designed_rush,td_designed_rush,fumbles_designed_rush,fumbles_lost_designed_rush,fumbles_out_of_bounds_designed_rush,epa_designed_rush,attempts_scramble_rush,yards_scramble_rush,td_scramble_rush,fumbles_scramble_rush,fumbles_lost_scramble_rush,fumbles_out_of_bounds_scramble_rush,epa_scramble_rush,attempts_kneel_rush,yards_kneel_rush,td_kneel_rush,fumbles_kneel_rush,fumbles_lost_kneel_rush,fumbles_out_of_bounds_kneel_rush,epa_kneel_rush,attempts_rush,yards_rush,td_rush,fumbles_rush,fumbles_lost_rush,fumbles_out_of_bounds_rush,epa_rush
0,2020,POST,2020_18_BAL_TEN,BAL,TEN,18,00-0034184,RB,G.Edwards,8,38,0,0,0,0,0.691066,8,38,0,0,0,0,0.691066,0,0,0,0,0,0,0.000000,0,0,0,0,0,0,0.000000,8,38,0,0,0,0,0.691066
1,2020,POST,2020_18_BAL_TEN,BAL,TEN,18,00-0034796,QB,L.Jackson,16,136,1,0,0,0,3.925763,13,90,0,0,0,0,1.531735,1,48,1,0,0,0,5.689723,2,-2,0,0,0,0,-3.295695,14,138,1,0,0,0,7.221458
2,2020,POST,2020_18_BAL_TEN,BAL,TEN,18,00-0035662,WR,M.Brown,2,19,0,0,0,0,1.412207,2,19,0,0,0,0,1.412207,0,0,0,0,0,0,0.000000,0,0,0,0,0,0,0.000000,2,19,0,0,0,0,1.412207
3,2020,POST,2020_18_BAL_TEN,BAL,TEN,18,00-0036158,RB,J.Dobbins,9,43,1,0,0,0,0.349493,9,43,1,0,0,0,0.349493,0,0,0,0,0,0,0.000000,0,0,0,0,0,0,0.000000,9,43,1,0,0,0,0.349493
4,2020,POST,2020_18_BAL_TEN,TEN,BAL,18,00-0029701,QB,R.Tannehill,2,6,0,0,0,0,1.980835,1,2,0,0,0,0,2.028814,1,4,0,0,0,0,-0.047978,0,0,0,0,0,0,0.000000,2,6,0,0,0,0,1.980835


In [235]:
"""
Columns to keep in the summary:
rec - receptions, targets, yards, air_yards_intended, air_yards_completed, fumbles_lost, epa, cpoe
"""

query = """SELECT * FROM receiving_by_player_by_game"""
df_rec = pd.read_sql(query, db_conn)
df_rec = prepare_columns(df_rec, "_rec")
df_rec.head()

,year,season_type,game_id,team,opp,week,gsis_id,pos,player,receptions_rec,targets_rec,yards_rec,air_yards_intended_rec,air_yards_completed_rec,td_rec,int_rec,fumbles_rec,fumbles_lost_rec,epa_rec,cpoe_rec
0,2020,REG,2020_01_ARI_SF,ARI,SF,1,00-0022921,WR,L.Fitzgerald,4,5,34,15,11,0,0,0,0,1.358564,17.277747
1,2020,REG,2020_01_ARI_SF,SF,ARI,1,00-0029892,FB,K.Juszczyk,1,2,41,20,18,0,0,0,0,2.172536,-39.573061
2,2020,REG,2020_01_ARI_SF,SF,ARI,1,00-0030472,TE,J.Reed,2,2,12,5,5,0,0,0,0,0.323281,39.810020
3,2020,REG,2020_01_ARI_SF,ARI,SF,1,00-0030564,WR,D.Hopkins,14,16,151,103,84,0,0,0,0,10.086382,267.857954
4,2020,REG,2020_01_ARI_SF,SF,ARI,1,00-0031376,RB,J.McKinnon,3,5,20,29,3,1,0,0,0,1.022131,-52.377328


In [236]:
df_rec[df_rec['player'] == 'B.Mayfield']

,year,season_type,game_id,team,opp,week,gsis_id,pos,player,receptions_rec,targets_rec,yards_rec,air_yards_intended_rec,air_yards_completed_rec,td_rec,int_rec,fumbles_rec,fumbles_lost_rec,epa_rec,cpoe_rec
2967,2020,REG,2020_13_CLE_TEN,CLE,TEN,13,00-0034855,QB,B.Mayfield,1,1,6,5,5,0,0,0,0,1.770588,48.384011
4343,2020,POST,2020_19_CLE_KC,CLE,KC,19,00-0034855,QB,B.Mayfield,1,1,-8,-9,-9,0,0,0,0,-1.111564,42.031807


In [237]:
query = """SELECT * FROM passing_by_team_by_game"""
df_pass_team = pd.read_sql(query, db_conn)
df_pass_team = df_pass_team[[col for col in df_pass_team.columns if not "_share" in col]]
df_pass_team = prepare_columns(df_pass_team, "_pass_team")
df_pass_team.head()

,year,season_type,game_id,team,opp,week,completions_pass_team,attempts_pass_team,yards_pass_team,air_yards_intended_pass_team,air_yards_completed_pass_team,td_pass_team,int_pass_team,sacks_pass_team,sack_yards_pass_team,fumbles_pass_team,fumbles_lost_pass_team,spikes_pass_team,epa_pass_team,epa_pass_pass_team,epa_int_pass_team,epa_sack_pass_team,epa_spike_pass_team,cpoe_pass_team,completions_wr_pass_team,attempts_wr_pass_team,yards_wr_pass_team,air_yards_intended_wr_pass_team,air_yards_completed_wr_pass_team,td_wr_pass_team,int_wr_pass_team,epa_wr_pass_team,cpoe_wr_pass_team,completions_te_pass_team,attempts_te_pass_team,yards_te_pass_team,air_yards_intended_te_pass_team,air_yards_completed_te_pass_team,td_te_pass_team,int_te_pass_team,epa_te_pass_team,cpoe_te_pass_team,completions_rb_pass_team,attempts_rb_pass_team,yards_rb_pass_team,air_yards_intended_rb_pass_team,air_yards_completed_rb_pass_team,td_rb_pass_team,int_rb_pass_team,epa_rb_pass_team,cpoe_rb_pass_team,completions_null_pass_team,attempts_null_pass_team,yards_null_pass_team,air_yards_intended_null_pass_team,air_yards_completed_null_pass_team,td_null_pass_team,int_null_pass_team,epa_null_pass_team,cpoe_null_pass_team,completions_other_pass_team,attempts_other_pass_team,yards_other_pass_team,air_yards_intended_other_pass_team,air_yards_completed_other_pass_team,td_other_pass_team,int_other_pass_team,epa_other_pass_team,cpoe_other_pass_team
0,2020,POST,2020_18_BAL_TEN,BAL,TEN,18,17,24,179,225,106,0,1,5,-14,0,0,0,1.736636,5.309047,-3.131832,-3.572411,0.0,5.980015,9,13,118,177,79,0,1,3.158207,117.877969,4,6,41,49,28,0,0,1.075494,45.480016,1,1,-6,-6,-6,0,0,-0.813038,13.602626,0,0,0,0,0,0,0,0.000000,0.0,3,4,26,5,5,0,0,1.888384,-3.540176
1,2020,POST,2020_18_BAL_TEN,TEN,BAL,18,18,26,165,188,95,1,1,1,-7,0,0,0,-0.451035,1.671480,-2.703767,-2.122515,0.0,5.348401,9,16,106,173,81,1,1,1.816069,-38.377932,5,6,58,22,21,0,0,1.950309,40.199798,3,3,11,3,3,0,0,-0.975724,90.209723,0,0,0,0,0,0,0,0.000000,0.0,1,1,-10,-10,-10,0,0,-1.119174,52.375245
2,2020,POST,2020_18_CHI_NO,CHI,NO,18,19,29,199,278,141,1,0,1,-8,0,0,0,-2.489986,-0.224911,0.000000,-2.265074,0.0,2.491256,14,20,158,238,119,0,0,0.649834,103.694338,5,6,41,22,22,1,0,1.610575,48.050398,0,1,0,4,0,0,0,-1.329776,-77.007067,0,2,0,14,0,0,0,-1.155545,0.0,0,0,0,0,0,0,0,0.000000,0.000000
3,2020,POST,2020_18_CHI_NO,NO,CHI,18,28,39,265,185,105,2,0,1,-3,1,1,0,8.355172,14.463355,0.000000,-6.108182,0.0,2.918653,15,18,173,112,74,1,0,17.097956,165.717143,5,10,44,70,28,0,0,-2.579102,-202.846012,4,4,28,-2,-2,1,0,3.826054,80.981386,0,3,0,0,0,0,0,-3.787438,0.0,4,4,20,5,5,0,0,-0.094116,69.974941
4,2020,POST,2020_18_CLE_PIT,CLE,PIT,18,21,34,263,231,93,3,0,0,0,0,0,0,14.296387,14.296387,0.000000,0.000000,0.0,-6.742178,8,17,128,142,55,1,0,5.139123,-341.385496,8,12,53,96,45,1,0,1.913639,27.971116,5,5,82,-7,-7,1,0,7.243625,84.180331,0,0,0,0,0,0,0,0.000000,0.0,0,0,0,0,0,0,0,0.000000,0.000000


In [238]:
query = """SELECT * FROM rushing_by_team_by_game"""
df_rush_team = pd.read_sql(query, db_conn)
df_rush_team = df_rush_team[[col for col in df_rush_team.columns if not "_share" in col]]
df_rush_team = prepare_columns(df_rush_team, "_rush_team")
df_rush_team.head()

,year,season_type,game_id,team,opp,week,attempts_total_rush_team,yards_total_rush_team,td_total_rush_team,fumbles_total_rush_team,fumbles_lost_total_rush_team,fumbles_out_of_bounds_total_rush_team,epa_total_rush_team,attempts_designed_rush_team,yards_designed_rush_team,td_designed_rush_team,fumbles_designed_rush_team,fumbles_lost_designed_rush_team,fumbles_out_of_bounds_designed_rush_team,epa_designed_rush_team,attempts_scramble_rush_team,yards_scramble_rush_team,td_scramble_rush_team,fumbles_scramble_rush_team,fumbles_lost_scramble_rush_team,fumbles_out_of_bounds_scramble_rush_team,epa_scramble_rush_team,attempts_kneel_rush_team,yards_kneel_rush_team,td_kneel_rush_team,fumbles_kneel_rush_team,fumbles_lost_kneel_rush_team,fumbles_out_of_bounds_kneel_rush_team,epa_kneel_rush_team,attempts_rush_team,yards_rush_team,td_rush_team,fumbles_rush_team,fumbles_lost_rush_team,fumbles_out_of_bounds_rush_team,epa_rush_team,attempts_rb_rush_team,yards_rb_rush_team,td_rb_rush_team,epa_rb_rush_team,attempts_qb_designed_rush_team,yards_qb_designed_rush_team,td_qb_designed_rush_team,epa_qb_designed_rush_team,attempts_qb_scramble_rush_team,yards_qb_scramble_rush_team,td_qb_scramble_rush_team,epa_qb_scramble_rush_team,attempts_qb_kneel_rush_team,yards_qb_kneel_rush_team,epa_qb_kneel_rush_team,attempts_wr_rush_team,yards_wr_rush_team,td_wr_rush_team,epa_wr_rush_team,attempts_other_rush_team,yards_other_rush_team,td_other_rush_team,epa_other_rush_team
0,2020,POST,2020_18_BAL_TEN,BAL,TEN,18,35,236,2,0,0,0,6.378529,32,190,1,0,0,0,3.984501,1,48,1,0,0,0,5.689723,2,-2,0,0,0,0,-3.295695,33,238,2,0,0,0,9.674223,17,81,1,1.040559,13,90,0,1.531735,1,48,1,5.689723,2,-2,-3.295695,2,19,0,1.412207,0,0,0,0.0
1,2020,POST,2020_18_BAL_TEN,TEN,BAL,18,22,51,0,0,0,0,-4.666149,21,47,0,0,0,0,-4.618171,1,4,0,0,0,0,-0.047978,0,0,0,0,0,0,0.000000,22,51,0,0,0,0,-4.666149,20,45,0,-6.646984,1,2,0,2.028814,1,4,0,-0.047978,0,0,0.000000,0,0,0,0.000000,0,0,0,0.0
2,2020,POST,2020_18_CHI_NO,CHI,NO,18,19,48,0,0,0,0,-8.719436,16,38,0,0,0,0,-5.140684,3,10,0,0,0,0,-3.578752,0,0,0,0,0,0,0.000000,19,48,0,0,0,0,-8.719436,13,31,0,-4.083455,0,0,0,0.000000,3,10,0,-3.578752,0,0,0.000000,3,7,0,-1.057229,0,0,0,0.0
3,2020,POST,2020_18_CHI_NO,NO,CHI,18,35,123,1,1,0,0,-8.546762,35,123,1,1,0,0,-8.546762,0,0,0,0,0,0,0.000000,0,0,0,0,0,0,0.000000,35,123,1,1,0,0,-8.546762,27,108,1,-2.411689,7,17,0,-4.792657,0,0,0,0.000000,0,0,0.000000,1,-2,0,-1.342416,0,0,0,0.0
4,2020,POST,2020_18_CLE_PIT,CLE,PIT,18,31,127,2,0,0,0,-1.390041,26,124,2,0,0,0,0.798019,2,8,0,0,0,0,-0.131966,3,-5,0,0,0,0,-2.056094,28,132,2,0,0,0,0.666054,26,124,2,0.798019,0,0,0,0.000000,2,8,0,-0.131966,3,-5,-2.056094,0,0,0,0.000000,0,0,0,0.0


In [239]:
query = """SELECT * FROM two_point_conversions_by_player_by_game"""
df_two_pt = pd.read_sql(query, db_conn)
df_two_pt.head()

,year,season_type,game_id,team,opp,week,gsis_id,pos,player,two_point_conversions
0,2020,POST,2020_18_CLE_PIT,PIT,CLE,18,00-0022924,QB,B.Roethlisberger,1
1,2020,POST,2020_18_IND_BUF,IND,BUF,18,00-0022942,QB,P.Rivers,1
2,2020,POST,2020_19_LA_GB,LA,GB,19,00-0033106,QB,J.Goff,1
3,2020,REG,2020_01_GB_MIN,MIN,GB,1,00-0029604,QB,K.Cousins,1
4,2020,REG,2020_01_LV_CAR,CAR,LV,1,00-0031237,QB,T.Bridgewater,1


In [240]:
join_cols = ['year', 'season_type', 'game_id', 'team', 'week','opp', 'gsis_id', 'pos', 'player']

df = df_pass.merge(df_rush, how='outer', on=join_cols)
df.head()

,year,season_type,game_id,team,opp,week,gsis_id,pos,player,completions_pass,attempts_pass,yards_pass,air_yards_intended_pass,air_yards_completed_pass,td_pass,int_pass,sacks_pass,sack_yards_pass,fumbles_pass,fumbles_lost_pass,spikes_pass,epa_pass,epa_pass_pass,epa_int_pass,epa_sack_pass,epa_spike_pass,cpoe_pass,completions_wr_pass,attempts_wr_pass,yards_wr_pass,air_yards_intended_wr_pass,air_yards_completed_wr_pass,td_wr_pass,int_wr_pass,epa_wr_pass,cpoe_wr_pass,completions_te_pass,attempts_te_pass,yards_te_pass,air_yards_intended_te_pass,air_yards_completed_te_pass,td_te_pass,int_te_pass,epa_te_pass,cpoe_te_pass,completions_rb_pass,attempts_rb_pass,yards_rb_pass,air_yards_intended_rb_pass,air_yards_completed_rb_pass,td_rb_pass,int_rb_pass,epa_rb_pass,cpoe_rb_pass,completions_null_pass,attempts_null_pass,yards_null_pass,air_yards_intended_null_pass,air_yards_completed_null_pass,td_null_pass,int_null_pass,epa_null_pass,cpoe_null_pass,completions_other_pass,attempts_other_pass,yards_other_pass,air_yards_intended_other_pass,air_yards_completed_other_pass,td_other_pass,int_other_pass,epa_other_pass,cpoe_other_pass,target_share_wr_pass,target_share_te_pass,target_share_rb_pass,target_share_other_pass,air_yards_intended_share_wr_pass,air_yards_intended_share_te_pass,air_yards_intended_share_rb_pass,air_yards_completed_share_wr_pass,air_yards_completed_share_te_pass,air_yards_completed_share_rb_pass,attempts_total_rush,yards_total_rush,td_total_rush,fumbles_total_rush,fumbles_lost_total_rush,fumbles_out_of_bounds_total_rush,epa_total_rush,attempts_designed_rush,yards_designed_rush,td_designed_rush,fumbles_designed_rush,fumbles_lost_designed_rush,fumbles_out_of_bounds_designed_rush,epa_designed_rush,attempts_scramble_rush,yards_scramble_rush,td_scramble_rush,fumbles_scramble_rush,fumbles_lost_scramble_rush,fumbles_out_of_bounds_scramble_rush,epa_scramble_rush,attempts_kneel_rush,yards_kneel_rush,td_kneel_rush,fumbles_kneel_rush,fumbles_lost_kneel_rush,fumbles_out_of_bounds_kneel_rush,epa_kneel_rush,attempts_rush,yards_rush,td_rush,fumbles_rush,fumbles_lost_rush,fumbles_out_of_bounds_rush,epa_rush
0,2020,REG,2020_03_KC_BAL,KC,BAL,3,00-0033873,QB,P.Mahomes,31.0,42.0,385.0,346.0,207.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,31.377849,31.377849,0.0,0.000000,0.000000,9.176690,16.0,21.0,220.0,288.0,169.0,2.0,0.0,18.443355,393.802585,6.0,8.0,87.0,52.0,38.0,0.0,0.0,5.017431,9.524000,8.0,11.0,76.0,1.0,-1.0,1.0,0.0,7.776404,-49.395210,0.0,1.0,0.0,4.0,0.0,0.0,0.0,-0.764009,0.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,0.904668,31.489611,0.500000,0.190476,0.261905,0.02381,0.832370,0.150289,0.002890,0.816425,0.183575,-0.004831,4.0,26.0,1.0,0.0,0.0,0.0,3.874581,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4.0,26.0,1.0,0.0,0.0,0.0,3.874581,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4.0,26.0,1.0,0.0,0.0,0.0,3.874581
1,2020,REG,2020_06_ATL_MIN,ATL,MIN,6,00-0026143,QB,M.Ryan,30.0,40.0,371.0,270.0,200.0,4.0,0.0,2.0,-8.0,0.0,0.0,0.0,30.441585,33.212461,0.0,-2.770877,0.000000,8.848381,19.0,25.0,268.0,224.0,170.0,3.0,0.0,23.358781,224.087441,4.0,4.0,57.0,21.0,21.0,1.0,0.0,8.707816,108.141237,6.0,7.0,43.0,5.0,6.0,0.0,0.0,3.862365,19.846362,0.0,3.0,0.0,17.0,0.0,0.0,0.0,-2.419047,0.0,1.0,1.0,3.0,3.0,3.0,0.0,0.0,-0.297454,19.556969,0.625000,0.100000,0.175000,0.02500,0.829630,0.077778,0.018519,0.850000,0.105000,0.030000,4.0,8.0,0.0,0.0,0.0,0.0,-2.871282,1.0,-4.0,0.0,0.0,0.0,0.0,-0.992540,2.0,13.0,0.0,0.0,0.0,0.0,-0.178477,1.0,-1.0,0.0,0.0,0.0,0.0,-1.700265,3.0,9.0,0.0,0.0,0.0,0.0,-1.171017
2,2020,REG,2020_08_NYJ_KC,KC,NYJ,8,00-0033873,QB,P.Mahomes,31.0,42.0,416.0,388.0,218.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,27.690373,27.690373,0.0,0.000000,0.000000,8.220293,14.0,20.0,249.0,294.0,171.0,4.0,0.0,17.918665,220.967728,10.0,15.0,119.0,105.0,58.0,1.0,0.0,6.302816,8.920118,7.0,7.0,48.0,-11.0,-11.0,0.0,0.0,3.468892,115.364480,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.476190,0.357143,0.166667,0.00000,0.757732,0.270619,-0.028351,0.784404,0.266055,-0.050459,NaN,NaN,NaN,NaN,NaN,Na

In [241]:
df = df.merge(df_rec, how='outer', on=join_cols)
df.head()

,year,season_type,game_id,team,opp,week,gsis_id,pos,player,completions_pass,attempts_pass,yards_pass,air_yards_intended_pass,air_yards_completed_pass,td_pass,int_pass,sacks_pass,sack_yards_pass,fumbles_pass,fumbles_lost_pass,spikes_pass,epa_pass,epa_pass_pass,epa_int_pass,epa_sack_pass,epa_spike_pass,cpoe_pass,completions_wr_pass,attempts_wr_pass,yards_wr_pass,air_yards_intended_wr_pass,air_yards_completed_wr_pass,td_wr_pass,int_wr_pass,epa_wr_pass,cpoe_wr_pass,completions_te_pass,attempts_te_pass,yards_te_pass,air_yards_intended_te_pass,air_yards_completed_te_pass,td_te_pass,int_te_pass,epa_te_pass,cpoe_te_pass,completions_rb_pass,attempts_rb_pass,yards_rb_pass,air_yards_intended_rb_pass,air_yards_completed_rb_pass,td_rb_pass,int_rb_pass,epa_rb_pass,cpoe_rb_pass,completions_null_pass,attempts_null_pass,yards_null_pass,air_yards_intended_null_pass,air_yards_completed_null_pass,td_null_pass,int_null_pass,epa_null_pass,cpoe_null_pass,completions_other_pass,attempts_other_pass,yards_other_pass,air_yards_intended_other_pass,air_yards_completed_other_pass,td_other_pass,int_other_pass,epa_other_pass,cpoe_other_pass,target_share_wr_pass,target_share_te_pass,target_share_rb_pass,target_share_other_pass,air_yards_intended_share_wr_pass,air_yards_intended_share_te_pass,air_yards_intended_share_rb_pass,air_yards_completed_share_wr_pass,air_yards_completed_share_te_pass,air_yards_completed_share_rb_pass,attempts_total_rush,yards_total_rush,td_total_rush,fumbles_total_rush,fumbles_lost_total_rush,fumbles_out_of_bounds_total_rush,epa_total_rush,attempts_designed_rush,yards_designed_rush,td_designed_rush,fumbles_designed_rush,fumbles_lost_designed_rush,fumbles_out_of_bounds_designed_rush,epa_designed_rush,attempts_scramble_rush,yards_scramble_rush,td_scramble_rush,fumbles_scramble_rush,fumbles_lost_scramble_rush,fumbles_out_of_bounds_scramble_rush,epa_scramble_rush,attempts_kneel_rush,yards_kneel_rush,td_kneel_rush,fumbles_kneel_rush,fumbles_lost_kneel_rush,fumbles_out_of_bounds_kneel_rush,epa_kneel_rush,attempts_rush,yards_rush,td_rush,fumbles_rush,fumbles_lost_rush,fumbles_out_of_bounds_rush,epa_rush,receptions_rec,targets_rec,yards_rec,air_yards_intended_rec,air_yards_completed_rec,td_rec,int_rec,fumbles_rec,fumbles_lost_rec,epa_rec,cpoe_rec
0,2020,REG,2020_03_KC_BAL,KC,BAL,3,00-0033873,QB,P.Mahomes,31.0,42.0,385.0,346.0,207.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,31.377849,31.377849,0.0,0.000000,0.000000,9.176690,16.0,21.0,220.0,288.0,169.0,2.0,0.0,18.443355,393.802585,6.0,8.0,87.0,52.0,38.0,0.0,0.0,5.017431,9.524000,8.0,11.0,76.0,1.0,-1.0,1.0,0.0,7.776404,-49.395210,0.0,1.0,0.0,4.0,0.0,0.0,0.0,-0.764009,0.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,0.904668,31.489611,0.500000,0.190476,0.261905,0.02381,0.832370,0.150289,0.002890,0.816425,0.183575,-0.004831,4.0,26.0,1.0,0.0,0.0,0.0,3.874581,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4.0,26.0,1.0,0.0,0.0,0.0,3.874581,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4.0,26.0,1.0,0.0,0.0,0.0,3.874581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,REG,2020_06_ATL_MIN,ATL,MIN,6,00-0026143,QB,M.Ryan,30.0,40.0,371.0,270.0,200.0,4.0,0.0,2.0,-8.0,0.0,0.0,0.0,30.441585,33.212461,0.0,-2.770877,0.000000,8.848381,19.0,25.0,268.0,224.0,170.0,3.0,0.0,23.358781,224.087441,4.0,4.0,57.0,21.0,21.0,1.0,0.0,8.707816,108.141237,6.0,7.0,43.0,5.0,6.0,0.0,0.0,3.862365,19.846362,0.0,3.0,0.0,17.0,0.0,0.0,0.0,-2.419047,0.0,1.0,1.0,3.0,3.0,3.0,0.0,0.0,-0.297454,19.556969,0.625000,0.100000,0.175000,0.02500,0.829630,0.077778,0.018519,0.850000,0.105000,0.030000,4.0,8.0,0.0,0.0,0.0,0.0,-2.871282,1.0,-4.0,0.0,0.0,0.0,0.0,-0.992540,2.0,13.0,0.0,0.0,0.0,0.0,-0.178477,1.0,-1.0,0.0,0.0,0.0,0.0,-1.700265,3.0,9.0,0.0,0.0,0.0,0.0,-1.171017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,REG,2020_08_NYJ_KC,KC,NYJ,8,00-0033873,QB,P.Mahomes,31.0,42.0,416.0,388.0,218.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,27.690373,27.690373,0.0,0.000000,0.000000,8.220293,14.0,20.0,249.0,294.0,171.0,4.0,0.0,17.918665,220.967728,10.0,15.0,119.0,105.0,58.0,1.0,0.0,6.302816,8.920118,7.0,7.0,48.0,-11.0,-

In [242]:
team_join_cols = ['year', 'season_type', 'game_id', 'team', 'opp', 'week']
df = df.merge(df_pass_team, how='outer', on=team_join_cols)
df.head()

,year,season_type,game_id,team,opp,week,gsis_id,pos,player,completions_pass,attempts_pass,yards_pass,air_yards_intended_pass,air_yards_completed_pass,td_pass,int_pass,sacks_pass,sack_yards_pass,fumbles_pass,fumbles_lost_pass,spikes_pass,epa_pass,epa_pass_pass,epa_int_pass,epa_sack_pass,epa_spike_pass,cpoe_pass,completions_wr_pass,attempts_wr_pass,yards_wr_pass,air_yards_intended_wr_pass,air_yards_completed_wr_pass,td_wr_pass,int_wr_pass,epa_wr_pass,cpoe_wr_pass,completions_te_pass,attempts_te_pass,yards_te_pass,air_yards_intended_te_pass,air_yards_completed_te_pass,td_te_pass,int_te_pass,epa_te_pass,cpoe_te_pass,completions_rb_pass,attempts_rb_pass,yards_rb_pass,air_yards_intended_rb_pass,air_yards_completed_rb_pass,td_rb_pass,int_rb_pass,epa_rb_pass,cpoe_rb_pass,completions_null_pass,attempts_null_pass,yards_null_pass,air_yards_intended_null_pass,air_yards_completed_null_pass,td_null_pass,int_null_pass,epa_null_pass,cpoe_null_pass,completions_other_pass,attempts_other_pass,yards_other_pass,air_yards_intended_other_pass,air_yards_completed_other_pass,td_other_pass,int_other_pass,epa_other_pass,cpoe_other_pass,target_share_wr_pass,target_share_te_pass,target_share_rb_pass,target_share_other_pass,air_yards_intended_share_wr_pass,air_yards_intended_share_te_pass,air_yards_intended_share_rb_pass,air_yards_completed_share_wr_pass,air_yards_completed_share_te_pass,air_yards_completed_share_rb_pass,attempts_total_rush,yards_total_rush,td_total_rush,fumbles_total_rush,fumbles_lost_total_rush,fumbles_out_of_bounds_total_rush,epa_total_rush,attempts_designed_rush,yards_designed_rush,td_designed_rush,fumbles_designed_rush,fumbles_lost_designed_rush,fumbles_out_of_bounds_designed_rush,epa_designed_rush,attempts_scramble_rush,yards_scramble_rush,td_scramble_rush,fumbles_scramble_rush,fumbles_lost_scramble_rush,fumbles_out_of_bounds_scramble_rush,epa_scramble_rush,attempts_kneel_rush,yards_kneel_rush,td_kneel_rush,fumbles_kneel_rush,fumbles_lost_kneel_rush,fumbles_out_of_bounds_kneel_rush,epa_kneel_rush,attempts_rush,yards_rush,td_rush,fumbles_rush,fumbles_lost_rush,fumbles_out_of_bounds_rush,epa_rush,receptions_rec,targets_rec,yards_rec,air_yards_intended_rec,air_yards_completed_rec,td_rec,int_rec,fumbles_rec,fumbles_lost_rec,epa_rec,cpoe_rec,completions_pass_team,attempts_pass_team,yards_pass_team,air_yards_intended_pass_team,air_yards_completed_pass_team,td_pass_team,int_pass_team,sacks_pass_team,sack_yards_pass_team,fumbles_pass_team,fumbles_lost_pass_team,spikes_pass_team,epa_pass_team,epa_pass_pass_team,epa_int_pass_team,epa_sack_pass_team,epa_spike_pass_team,cpoe_pass_team,completions_wr_pass_team,attempts_wr_pass_team,yards_wr_pass_team,air_yards_intended_wr_pass_team,air_yards_completed_wr_pass_team,td_wr_pass_team,int_wr_pass_team,epa_wr_pass_team,cpoe_wr_pass_team,completions_te_pass_team,attempts_te_pass_team,yards_te_pass_team,air_yards_intended_te_pass_team,air_yards_completed_te_pass_team,td_te_pass_team,int_te_pass_team,epa_te_pass_team,cpoe_te_pass_team,completions_rb_pass_team,attempts_rb_pass_team,yards_rb_pass_team,air_yards_intended_rb_pass_team,air_yards_completed_rb_pass_team,td_rb_pass_team,int_rb_pass_team,epa_rb_pass_team,cpoe_rb_pass_team,completions_null_pass_team,attempts_null_pass_team,yards_null_pass_team,air_yards_intended_null_pass_team,air_yards_completed_null_pass_team,td_null_pass_team,int_null_pass_team,epa_null_pass_team,cpoe_null_pass_team,completions_other_pass_team,attempts_other_pass_team,yards_other_pass_team,air_yards_intended_other_pass_team,air_yards_completed_other_pass_team,td_other_pass_team,int_other_pass_team,epa_other_pass_team,cpoe_other_pass_team
0,2020,REG,2020_03_KC_BAL,KC,BAL,3,00-0033873,QB,P.Mahomes,31.0,42.0,385.0,346.0,207.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,31.377849,31.377849,0.0,0.0,0.0,9.17669,16.0,21.0,220.0,288.0,169.0,2.0,0.0,18.443355,393.802585,6.0,8.0,87.0,52.0,38.0,0.0,0.0,5.017431,9.524,8.0,11.0,76.0,1.0,-1.0,1.0,0.0,7.776404,-49.39521,0.0,1.0,0.0,4.0,0.0,0.0,0.0,-0.764009,0

In [243]:
df = df.merge(df_rush_team, how='outer', on=team_join_cols)
df.head()

,year,season_type,game_id,team,opp,week,gsis_id,pos,player,completions_pass,attempts_pass,yards_pass,air_yards_intended_pass,air_yards_completed_pass,td_pass,int_pass,sacks_pass,sack_yards_pass,fumbles_pass,fumbles_lost_pass,spikes_pass,epa_pass,epa_pass_pass,epa_int_pass,epa_sack_pass,epa_spike_pass,cpoe_pass,completions_wr_pass,attempts_wr_pass,yards_wr_pass,air_yards_intended_wr_pass,air_yards_completed_wr_pass,td_wr_pass,int_wr_pass,epa_wr_pass,cpoe_wr_pass,completions_te_pass,attempts_te_pass,yards_te_pass,air_yards_intended_te_pass,air_yards_completed_te_pass,td_te_pass,int_te_pass,epa_te_pass,cpoe_te_pass,completions_rb_pass,attempts_rb_pass,yards_rb_pass,air_yards_intended_rb_pass,air_yards_completed_rb_pass,td_rb_pass,int_rb_pass,epa_rb_pass,cpoe_rb_pass,completions_null_pass,attempts_null_pass,yards_null_pass,air_yards_intended_null_pass,air_yards_completed_null_pass,td_null_pass,int_null_pass,epa_null_pass,cpoe_null_pass,completions_other_pass,attempts_other_pass,yards_other_pass,air_yards_intended_other_pass,air_yards_completed_other_pass,td_other_pass,int_other_pass,epa_other_pass,cpoe_other_pass,target_share_wr_pass,target_share_te_pass,target_share_rb_pass,target_share_other_pass,air_yards_intended_share_wr_pass,air_yards_intended_share_te_pass,air_yards_intended_share_rb_pass,air_yards_completed_share_wr_pass,air_yards_completed_share_te_pass,air_yards_completed_share_rb_pass,attempts_total_rush,yards_total_rush,td_total_rush,fumbles_total_rush,fumbles_lost_total_rush,fumbles_out_of_bounds_total_rush,epa_total_rush,attempts_designed_rush,yards_designed_rush,td_designed_rush,fumbles_designed_rush,fumbles_lost_designed_rush,fumbles_out_of_bounds_designed_rush,epa_designed_rush,attempts_scramble_rush,yards_scramble_rush,td_scramble_rush,fumbles_scramble_rush,...,air_yards_intended_wr_pass_team,air_yards_completed_wr_pass_team,td_wr_pass_team,int_wr_pass_team,epa_wr_pass_team,cpoe_wr_pass_team,completions_te_pass_team,attempts_te_pass_team,yards_te_pass_team,air_yards_intended_te_pass_team,air_yards_completed_te_pass_team,td_te_pass_team,int_te_pass_team,epa_te_pass_team,cpoe_te_pass_team,completions_rb_pass_team,attempts_rb_pass_team,yards_rb_pass_team,air_yards_intended_rb_pass_team,air_yards_completed_rb_pass_team,td_rb_pass_team,int_rb_pass_team,epa_rb_pass_team,cpoe_rb_pass_team,completions_null_pass_team,attempts_null_pass_team,yards_null_pass_team,air_yards_intended_null_pass_team,air_yards_completed_null_pass_team,td_null_pass_team,int_null_pass_team,epa_null_pass_team,cpoe_null_pass_team,completions_other_pass_team,attempts_other_pass_team,yards_other_pass_team,air_yards_intended_other_pass_team,air_yards_completed_other_pass_team,td_other_pass_team,int_other_pass_team,epa_other_pass_team,cpoe_other_pass_team,attempts_total_rush_team,yards_total_rush_team,td_total_rush_team,fumbles_total_rush_team,fumbles_lost_total_rush_team,fumbles_out_of_bounds_total_rush_team,epa_total_rush_team,attempts_designed_rush_team,yards_designed_rush_team,td_designed_rush_team,fumbles_designed_rush_team,fumbles_lost_designed_rush_team,fumbles_out_of_bounds_designed_rush_team,epa_designed_rush_team,attempts_scramble_rush_team,yards_scramble_rush_team,td_scramble_rush_team,fumbles_scramble_rush_team,fumbles_lost_scramble_rush_team,fumbles_out_of_bounds_scramble_rush_team,epa_scramble_rush_team,attempts_kneel_rush_team,yards_kneel_rush_team,td_kneel_rush_team,fumbles_kneel_rush_team,fumbles_lost_kneel_rush_team,fumbles_out_of_bounds_kneel_rush_team,epa_kneel_rush_team,attempts_rush_team,yards_rush_team,td_rush_team,fumbles_rush_team,fumbles_lost_rush_team,fumbles_out_of_bounds_rush_team,epa_rush_team,attempts_rb_rush_team,yards_rb_rush_team,td_rb_rush_team,epa_rb_rush_team,attempts_qb_designed_rush_team,yards_qb_designed_rush_team,td_qb_designed_rush_team,epa_qb_designed_rush_team,attempts_qb_scramble_rush_team,yards_qb_scramble_rush_team,td_qb_scramble_rush_team,epa_qb_scramble_rush_team,attempts_qb_kneel_rush_team,yards_qb_knee

In [244]:
df = df.merge(df_two_pt, how='outer', on=join_cols)
df.head()

,year,season_type,game_id,team,opp,week,gsis_id,pos,player,completions_pass,attempts_pass,yards_pass,air_yards_intended_pass,air_yards_completed_pass,td_pass,int_pass,sacks_pass,sack_yards_pass,fumbles_pass,fumbles_lost_pass,spikes_pass,epa_pass,epa_pass_pass,epa_int_pass,epa_sack_pass,epa_spike_pass,cpoe_pass,completions_wr_pass,attempts_wr_pass,yards_wr_pass,air_yards_intended_wr_pass,air_yards_completed_wr_pass,td_wr_pass,int_wr_pass,epa_wr_pass,cpoe_wr_pass,completions_te_pass,attempts_te_pass,yards_te_pass,air_yards_intended_te_pass,air_yards_completed_te_pass,td_te_pass,int_te_pass,epa_te_pass,cpoe_te_pass,completions_rb_pass,attempts_rb_pass,yards_rb_pass,air_yards_intended_rb_pass,air_yards_completed_rb_pass,td_rb_pass,int_rb_pass,epa_rb_pass,cpoe_rb_pass,completions_null_pass,attempts_null_pass,yards_null_pass,air_yards_intended_null_pass,air_yards_completed_null_pass,td_null_pass,int_null_pass,epa_null_pass,cpoe_null_pass,completions_other_pass,attempts_other_pass,yards_other_pass,air_yards_intended_other_pass,air_yards_completed_other_pass,td_other_pass,int_other_pass,epa_other_pass,cpoe_other_pass,target_share_wr_pass,target_share_te_pass,target_share_rb_pass,target_share_other_pass,air_yards_intended_share_wr_pass,air_yards_intended_share_te_pass,air_yards_intended_share_rb_pass,air_yards_completed_share_wr_pass,air_yards_completed_share_te_pass,air_yards_completed_share_rb_pass,attempts_total_rush,yards_total_rush,td_total_rush,fumbles_total_rush,fumbles_lost_total_rush,fumbles_out_of_bounds_total_rush,epa_total_rush,attempts_designed_rush,yards_designed_rush,td_designed_rush,fumbles_designed_rush,fumbles_lost_designed_rush,fumbles_out_of_bounds_designed_rush,epa_designed_rush,attempts_scramble_rush,yards_scramble_rush,td_scramble_rush,fumbles_scramble_rush,...,air_yards_completed_wr_pass_team,td_wr_pass_team,int_wr_pass_team,epa_wr_pass_team,cpoe_wr_pass_team,completions_te_pass_team,attempts_te_pass_team,yards_te_pass_team,air_yards_intended_te_pass_team,air_yards_completed_te_pass_team,td_te_pass_team,int_te_pass_team,epa_te_pass_team,cpoe_te_pass_team,completions_rb_pass_team,attempts_rb_pass_team,yards_rb_pass_team,air_yards_intended_rb_pass_team,air_yards_completed_rb_pass_team,td_rb_pass_team,int_rb_pass_team,epa_rb_pass_team,cpoe_rb_pass_team,completions_null_pass_team,attempts_null_pass_team,yards_null_pass_team,air_yards_intended_null_pass_team,air_yards_completed_null_pass_team,td_null_pass_team,int_null_pass_team,epa_null_pass_team,cpoe_null_pass_team,completions_other_pass_team,attempts_other_pass_team,yards_other_pass_team,air_yards_intended_other_pass_team,air_yards_completed_other_pass_team,td_other_pass_team,int_other_pass_team,epa_other_pass_team,cpoe_other_pass_team,attempts_total_rush_team,yards_total_rush_team,td_total_rush_team,fumbles_total_rush_team,fumbles_lost_total_rush_team,fumbles_out_of_bounds_total_rush_team,epa_total_rush_team,attempts_designed_rush_team,yards_designed_rush_team,td_designed_rush_team,fumbles_designed_rush_team,fumbles_lost_designed_rush_team,fumbles_out_of_bounds_designed_rush_team,epa_designed_rush_team,attempts_scramble_rush_team,yards_scramble_rush_team,td_scramble_rush_team,fumbles_scramble_rush_team,fumbles_lost_scramble_rush_team,fumbles_out_of_bounds_scramble_rush_team,epa_scramble_rush_team,attempts_kneel_rush_team,yards_kneel_rush_team,td_kneel_rush_team,fumbles_kneel_rush_team,fumbles_lost_kneel_rush_team,fumbles_out_of_bounds_kneel_rush_team,epa_kneel_rush_team,attempts_rush_team,yards_rush_team,td_rush_team,fumbles_rush_team,fumbles_lost_rush_team,fumbles_out_of_bounds_rush_team,epa_rush_team,attempts_rb_rush_team,yards_rb_rush_team,td_rb_rush_team,epa_rb_rush_team,attempts_qb_designed_rush_team,yards_qb_designed_rush_team,td_qb_designed_rush_team,epa_qb_designed_rush_team,attempts_qb_scramble_rush_team,yards_qb_scramble_rush_team,td_qb_scramble_rush_team,epa_qb_scramble_rush_team,attempts_qb_kneel_rush_team,yards_qb_kneel_rush_team,epa_qb_kneel_rush_te

In [245]:
df = df.fillna(0)

In [246]:
df[df['player'] == 'C.Davis']

,year,season_type,game_id,team,opp,week,gsis_id,pos,player,completions_pass,attempts_pass,yards_pass,air_yards_intended_pass,air_yards_completed_pass,td_pass,int_pass,sacks_pass,sack_yards_pass,fumbles_pass,fumbles_lost_pass,spikes_pass,epa_pass,epa_pass_pass,epa_int_pass,epa_sack_pass,epa_spike_pass,cpoe_pass,completions_wr_pass,attempts_wr_pass,yards_wr_pass,air_yards_intended_wr_pass,air_yards_completed_wr_pass,td_wr_pass,int_wr_pass,epa_wr_pass,cpoe_wr_pass,completions_te_pass,attempts_te_pass,yards_te_pass,air_yards_intended_te_pass,air_yards_completed_te_pass,td_te_pass,int_te_pass,epa_te_pass,cpoe_te_pass,completions_rb_pass,attempts_rb_pass,yards_rb_pass,air_yards_intended_rb_pass,air_yards_completed_rb_pass,td_rb_pass,int_rb_pass,epa_rb_pass,cpoe_rb_pass,completions_null_pass,attempts_null_pass,yards_null_pass,air_yards_intended_null_pass,air_yards_completed_null_pass,td_null_pass,int_null_pass,epa_null_pass,cpoe_null_pass,completions_other_pass,attempts_other_pass,yards_other_pass,air_yards_intended_other_pass,air_yards_completed_other_pass,td_other_pass,int_other_pass,epa_other_pass,cpoe_other_pass,target_share_wr_pass,target_share_te_pass,target_share_rb_pass,target_share_other_pass,air_yards_intended_share_wr_pass,air_yards_intended_share_te_pass,air_yards_intended_share_rb_pass,air_yards_completed_share_wr_pass,air_yards_completed_share_te_pass,air_yards_completed_share_rb_pass,attempts_total_rush,yards_total_rush,td_total_rush,fumbles_total_rush,fumbles_lost_total_rush,fumbles_out_of_bounds_total_rush,epa_total_rush,attempts_designed_rush,yards_designed_rush,td_designed_rush,fumbles_designed_rush,fumbles_lost_designed_rush,fumbles_out_of_bounds_designed_rush,epa_designed_rush,attempts_scramble_rush,yards_scramble_rush,td_scramble_rush,fumbles_scramble_rush,...,air_yards_completed_wr_pass_team,td_wr_pass_team,int_wr_pass_team,epa_wr_pass_team,cpoe_wr_pass_team,completions_te_pass_team,attempts_te_pass_team,yards_te_pass_team,air_yards_intended_te_pass_team,air_yards_completed_te_pass_team,td_te_pass_team,int_te_pass_team,epa_te_pass_team,cpoe_te_pass_team,completions_rb_pass_team,attempts_rb_pass_team,yards_rb_pass_team,air_yards_intended_rb_pass_team,air_yards_completed_rb_pass_team,td_rb_pass_team,int_rb_pass_team,epa_rb_pass_team,cpoe_rb_pass_team,completions_null_pass_team,attempts_null_pass_team,yards_null_pass_team,air_yards_intended_null_pass_team,air_yards_completed_null_pass_team,td_null_pass_team,int_null_pass_team,epa_null_pass_team,cpoe_null_pass_team,completions_other_pass_team,attempts_other_pass_team,yards_other_pass_team,air_yards_intended_other_pass_team,air_yards_completed_other_pass_team,td_other_pass_team,int_other_pass_team,epa_other_pass_team,cpoe_other_pass_team,attempts_total_rush_team,yards_total_rush_team,td_total_rush_team,fumbles_total_rush_team,fumbles_lost_total_rush_team,fumbles_out_of_bounds_total_rush_team,epa_total_rush_team,attempts_designed_rush_team,yards_designed_rush_team,td_designed_rush_team,fumbles_designed_rush_team,fumbles_lost_designed_rush_team,fumbles_out_of_bounds_designed_rush_team,epa_designed_rush_team,attempts_scramble_rush_team,yards_scramble_rush_team,td_scramble_rush_team,fumbles_scramble_rush_team,fumbles_lost_scramble_rush_team,fumbles_out_of_bounds_scramble_rush_team,epa_scramble_rush_team,attempts_kneel_rush_team,yards_kneel_rush_team,td_kneel_rush_team,fumbles_kneel_rush_team,fumbles_lost_kneel_rush_team,fumbles_out_of_bounds_kneel_rush_team,epa_kneel_rush_team,attempts_rush_team,yards_rush_team,td_rush_team,fumbles_rush_team,fumbles_lost_rush_team,fumbles_out_of_bounds_rush_team,epa_rush_team,attempts_rb_rush_team,yards_rb_rush_team,td_rb_rush_team,epa_rb_rush_team,attempts_qb_designed_rush_team,yards_qb_designed_rush_team,td_qb_designed_rush_team,epa_qb_designed_rush_team,attempts_qb_scramble_rush_team,yards_qb_scramble_rush_team,td_qb_scramble_rush_team,epa_qb_scramble_rush_team,attempts_qb_kneel_rush_team,yards_qb_kneel_rush_team,epa_qb_kneel_rush_te

In [259]:
df['target_share'] = df['targets_rec'] / df['attempts_pass_team']
df['air_yards_intended_share'] = df['air_yards_intended_rec'] / df['air_yards_intended_pass_team']
df['rush_attempt_share'] = df['attempts_rush'] / df['attempts_rush_team']
df['rush_yards_share'] = df['yards_rush'] / df['yards_rush_team']



def get_positional_target_share(row):
    pos = row['pos'].lower()
    if pos not in ('rb', 'wr', 'te'): 
        return None
    
    team_pos_targets = row[f"attempts_{pos}_pass_team"]
    if team_pos_targets == 0:
        return 0

    return row['targets_rec'] / team_pos_targets


def get_positional_air_yards_share(row):
    pos = row['pos'].lower()
    if pos not in ('rb', 'wr', 'te'): 
        return None
    
    team_pos_air_yards = row[f"air_yards_intended_{pos}_pass_team"]
    if team_pos_air_yards == 0:
        return None

    return row['air_yards_intended_rec'] / team_pos_air_yards


def get_positional_rush_yards_share(row)

df['target_share_pos'] = df.apply(get_positional_target_share, axis=1)
df['air_yards_intended_share_pos'] = df.apply(get_positional_air_yards_share, axis=1)

df[df['player'] == 'C.Davis'][join_cols + \
        ['target_share', 'target_share_pos', 'air_yards_intended_share', 'air_yards_intended_share_pos']]

SyntaxError: unexpected EOF while parsing (<ipython-input-259-c1a538da9b1f>, line 15)

In [249]:
df[df['player'] == 'J.Dobbins'][join_cols + ['rush_attempt_share', 'rush_yards_share']]

,year,season_type,game_id,team,week,opp,gsis_id,pos,player,rush_attempt_share,rush_yards_share
574,2020,REG,2020_15_JAX_BAL,BAL,15,JAX,00-0036158,RB,J.Dobbins,0.400000,0.397516
665,2020,REG,2020_01_CLE_BAL,BAL,1,CLE,00-0036158,RB,J.Dobbins,0.241379,0.196429
1164,2020,REG,2020_16_NYG_BAL,BAL,16,NYG,00-0036158,RB,J.Dobbins,0.282051,0.308000
1378,2020,REG,2020_04_BAL_WAS,BAL,4,WAS,00-0036158,RB,J.Dobbins,0.166667,0.109589
1811,2020,REG,2020_09_BAL_IND,BAL,9,IND,00-0036158,RB,J.Dobbins,0.315789,0.272727
1902,2020,REG,2020_17_BAL_CIN,BAL,17,CIN,00-0036158,RB,J.Dobbins,0.240741,0.396040
1920,2020,REG,2020_14_BAL_CLE,BAL,14,CLE,00-0036158,RB,J.Dobbins,0.406250,0.229437
2182,2020,REG,2020_02_BAL_HOU,BAL,2,HOU,00-0036158,RB,J.Dobbins,0.058824,0.206009
2193,2020,REG,2020_11_TEN_BAL,BAL,11,TEN,00-0036158,RB,J.Dobbins,0.454545,0.542636
2479,2020,REG,2020_10_BAL_NE,BAL,10,NE,00-0036158,RB,J.Dobbins,0.178571,0.113043


In [176]:
print([col for col in df.columns if "fumble" in col and 'team' not in col])

['fumbles_pass', 'fumbles_lost_pass', 'fumbles_total_rush', 'fumbles_lost_total_rush', 'fumbles_out_of_bounds_total_rush', 'fumbles_designed_rush', 'fumbles_lost_designed_rush', 'fumbles_out_of_bounds_designed_rush', 'fumbles_scramble_rush', 'fumbles_lost_scramble_rush', 'fumbles_out_of_bounds_scramble_rush', 'fumbles_kneel_rush', 'fumbles_lost_kneel_rush', 'fumbles_out_of_bounds_kneel_rush', 'fumbles_rush', 'fumbles_lost_rush', 'fumbles_out_of_bounds_rush', 'fumbles_rec', 'fumbles_lost_rec']


In [251]:
df['total_fumbles_lost'] = df['fumbles_lost_rush'] + df['fumbles_lost_rec'] + df['fumbles_lost_pass']

In [252]:

def get_fantasy_points(row, ppr=0.5):
    return row['td_pass'] * 4 \
        + row['yards_pass'] / 25 \
        + row['int_pass'] * -2 \
        + row['yards_rush'] / 10 \
        + row['td_rush'] * 6 \
        + row['total_fumbles_lost'] * -2 \
        + row['receptions_rec'] * ppr \
        + row['yards_rec'] / 10 \
        + row['td_rec'] * 6 \
        + row['two_point_conversions'] * 2

df['fantasy_pts_half'] = df.apply(get_fantasy_points, args=(.5,), axis=1)
df['fantasy_pts_full'] = df.apply(get_fantasy_points, args=(1,), axis=1)


In [253]:
df.head()

,year,season_type,game_id,team,opp,week,gsis_id,pos,player,completions_pass,attempts_pass,yards_pass,air_yards_intended_pass,air_yards_completed_pass,td_pass,int_pass,sacks_pass,sack_yards_pass,fumbles_pass,fumbles_lost_pass,spikes_pass,epa_pass,epa_pass_pass,epa_int_pass,epa_sack_pass,epa_spike_pass,cpoe_pass,completions_wr_pass,attempts_wr_pass,yards_wr_pass,air_yards_intended_wr_pass,air_yards_completed_wr_pass,td_wr_pass,int_wr_pass,epa_wr_pass,cpoe_wr_pass,completions_te_pass,attempts_te_pass,yards_te_pass,air_yards_intended_te_pass,air_yards_completed_te_pass,td_te_pass,int_te_pass,epa_te_pass,cpoe_te_pass,completions_rb_pass,attempts_rb_pass,yards_rb_pass,air_yards_intended_rb_pass,air_yards_completed_rb_pass,td_rb_pass,int_rb_pass,epa_rb_pass,cpoe_rb_pass,completions_null_pass,attempts_null_pass,yards_null_pass,air_yards_intended_null_pass,air_yards_completed_null_pass,td_null_pass,int_null_pass,epa_null_pass,cpoe_null_pass,completions_other_pass,attempts_other_pass,yards_other_pass,air_yards_intended_other_pass,air_yards_completed_other_pass,td_other_pass,int_other_pass,epa_other_pass,cpoe_other_pass,target_share_wr_pass,target_share_te_pass,target_share_rb_pass,target_share_other_pass,air_yards_intended_share_wr_pass,air_yards_intended_share_te_pass,air_yards_intended_share_rb_pass,air_yards_completed_share_wr_pass,air_yards_completed_share_te_pass,air_yards_completed_share_rb_pass,attempts_total_rush,yards_total_rush,td_total_rush,fumbles_total_rush,fumbles_lost_total_rush,fumbles_out_of_bounds_total_rush,epa_total_rush,attempts_designed_rush,yards_designed_rush,td_designed_rush,fumbles_designed_rush,fumbles_lost_designed_rush,fumbles_out_of_bounds_designed_rush,epa_designed_rush,attempts_scramble_rush,yards_scramble_rush,td_scramble_rush,fumbles_scramble_rush,...,air_yards_completed_te_pass_team,td_te_pass_team,int_te_pass_team,epa_te_pass_team,cpoe_te_pass_team,completions_rb_pass_team,attempts_rb_pass_team,yards_rb_pass_team,air_yards_intended_rb_pass_team,air_yards_completed_rb_pass_team,td_rb_pass_team,int_rb_pass_team,epa_rb_pass_team,cpoe_rb_pass_team,completions_null_pass_team,attempts_null_pass_team,yards_null_pass_team,air_yards_intended_null_pass_team,air_yards_completed_null_pass_team,td_null_pass_team,int_null_pass_team,epa_null_pass_team,cpoe_null_pass_team,completions_other_pass_team,attempts_other_pass_team,yards_other_pass_team,air_yards_intended_other_pass_team,air_yards_completed_other_pass_team,td_other_pass_team,int_other_pass_team,epa_other_pass_team,cpoe_other_pass_team,attempts_total_rush_team,yards_total_rush_team,td_total_rush_team,fumbles_total_rush_team,fumbles_lost_total_rush_team,fumbles_out_of_bounds_total_rush_team,epa_total_rush_team,attempts_designed_rush_team,yards_designed_rush_team,td_designed_rush_team,fumbles_designed_rush_team,fumbles_lost_designed_rush_team,fumbles_out_of_bounds_designed_rush_team,epa_designed_rush_team,attempts_scramble_rush_team,yards_scramble_rush_team,td_scramble_rush_team,fumbles_scramble_rush_team,fumbles_lost_scramble_rush_team,fumbles_out_of_bounds_scramble_rush_team,epa_scramble_rush_team,attempts_kneel_rush_team,yards_kneel_rush_team,td_kneel_rush_team,fumbles_kneel_rush_team,fumbles_lost_kneel_rush_team,fumbles_out_of_bounds_kneel_rush_team,epa_kneel_rush_team,attempts_rush_team,yards_rush_team,td_rush_team,fumbles_rush_team,fumbles_lost_rush_team,fumbles_out_of_bounds_rush_team,epa_rush_team,attempts_rb_rush_team,yards_rb_rush_team,td_rb_rush_team,epa_rb_rush_team,attempts_qb_designed_rush_team,yards_qb_designed_rush_team,td_qb_designed_rush_team,epa_qb_designed_rush_team,attempts_qb_scramble_rush_team,yards_qb_scramble_rush_team,td_qb_scramble_rush_team,epa_qb_scramble_rush_team,attempts_qb_kneel_rush_team,yards_qb_kneel_rush_team,epa_qb_kneel_rush_team,attempts_wr_rush_team,yards_wr_rush_team,td_wr_rush_team,epa_wr_rush_team,attempts_other_rush_team,yards_other_rush_team,td_other_rush_team,epa_other_rush_team,two_point_conversions,target_share,a

In [ ]:
df['epa_total']
df['epa_total_true'] ## remove spikes and kneels

In [254]:
"""
pass - completions, attempts, yards, air_yards_intended, air_yards_completed, td, int, sacks, sack_yards, fumbles,
fumbles_lost, spikes, epa, epa_pass, epa_int, epa_sack, epa_spike, cpoe

rush - attempts_totals, yards_totals, td, fumbles, fumbles_lost, epa_total,
attempts_designed, yards_designed, td_designed, epa_designed,
attempts_scrambles, yards_scrambles, td_scrambles,
attempts_kneels, yards_kneels, epa_kneels,
attempts, yards, epa

rec - receptions, targets, yards, air_yards_intended, air_yards_completed, fumbles_lost, epa, cpoe

summaries:
air_yard_intended_share, air_yard_completed_share, air_yard_intended_share_pos, air_yards_completed_share_pos,
target_share, target_share_pos, yard_share_pos, yards_share
rush_yards_share, rush_attempts_share, rush_yards_share_pos, rush_attempts_share_pos, 
epa_total, epa_total_adj, fantasy_points_half, fantasy_points_full
"""

keeper_columns = [
    # passing
    'completions_pass', 'attempts_pass', 'yards_pass', 'air_yards_intended_pass', 'air_yards_completed_pass',
    'td_pass', 'int_pass', 'sacks_pass', 'sack_yards_pass', 'fumbles_pass', 'fumbles_lost_pass', 'spikes',
    'epa_pass', 'epa_pass_pass', 'epa_int', 'epa_sack', 'epa_spike', 'cpoe',
    # rushing
    'attempts_rush', 'yards_rush', 'td_rush', 'fumbles_rush', 'fumbles_lost_rush', 'epa_total_rush', 
    'attempts_designed_rush', 'yards_designed_rush', 'td_designed_rush',
    'attempts_scramble_rush', 'yards_scramble_rush', 'td_scramble_rush',
    'attempts_kneels_rush', 'yards_kneels_rush', 'epa_kneels_rush', 'epa_rush',
    # receiving
    'receptions_rec', 'targets_rec', 'yards_rec', 'air_yards_intended_rec', 'air_yards_completed_rec',
    'fumbles_rec', 'fumbles_lost_rec', 'cpoe_rec'
    # summaries
    'air_yards_intended_share', 'air_yards_completed_share',
    'air_yards_intended_share_pos', 'air_yards_completed_share_pos',
    'target_share', 'target_share_pos', 'yards_rec_share', 'yards_rec_share_pos'
    'attempts_rush_share', 'yards_rush_share', 'attempts_rush_share_pos', 'yards_rush_share_pos',
    'fantasy_points_half', 'fantasy_points_full',
    'epa', 'epa_true'
    
]


Index(['year', 'season_type', 'game_id', 'team', 'opp', 'week', 'gsis_id',
       'pos', 'player', 'completions_pass',
       ...
       'two_point_conversions', 'target_share', 'air_yards_intended_share',
       'target_share_pos', 'air_yards_intended_share_pos',
       'rush_attempt_share', 'rush_yards_share', 'total_fumbles_lost',
       'fantasy_pts_half', 'fantasy_pts_full'],
      dtype='object', length=259)

In [255]:
len(df.columns)

259

259

## Adding 2 point conversions

In [198]:
query = """SELECT * FROM play_by_play_enriched where two_point_attempt = 1"""
df = pd.read_sql(query, db_conn)
df.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,...,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,stadium,weather,nfl_api_id,play_clock,play_deleted,play_type_nfl,special_teams_play,st_play_type,end_clock_time,end_yard_line,fixed_drive,fixed_drive_result,drive_real_start_time,drive_play_count,drive_time_of_possession,drive_first_downs,drive_inside20,drive_ended_with_score,drive_quarter_start,drive_quarter_end,drive_yards_penalized,drive_start_transition,drive_end_transition,drive_game_clock_start,drive_game_clock_end,drive_start_yard_line,drive_end_yard_line,drive_play_id_started,drive_play_id_ended,away_score,home_score,location,result,total,spread_line,total_line,div_game,roof,surface,temp,wind,home_coach,away_coach,stadium_id,game_stadium,success,passer,passer_jersey_number,rusher,rusher_jersey_number,receiver,receiver_jersey_number,pass,rush,first_down,aborted_play,special,play,passer_id,rusher_id,receiver_id,name,jersey_number,id,fantasy_player_name,fantasy_player_id,fantasy,fantasy_id,out_of_bounds,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe,year,passer_position,receiver_position,rusher_position
0,2922.0,2020_01_GB_MIN,2.020091e+09,MIN,GB,REG,1.0,MIN,home,GB,GB,2.0,2020-09-13,833.0,833.0,833.0,Half2,0,15.0,1,4.0,None,0,49980000000000,GB 2,0.0,77.0,(Pass formation) TWO-POINT CONVERSION ATTEMPT. 33-D.Cook rushes up the middle. ATTEMPT SUCCEEDS.,run,2.0,0,0,0.0,0,0,0,None,None,None,None,None,None,None,None,None,success,2.0,2.0,0.0,None,None,None,None,2.0,2.0,18.0,29.0,16.0,29.0,-13.0,18.0,29.0,-11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4735,0.947,1.053,-15.448462,15.448462,5.010020,-5.010020,-14.483307,14.483307,None,None,0.0,0.0,0.761230,-0.761230,-17.457853,17.457853,-6.289148,6.289148,-4.555583,4.555583,0.058003,0.941997,0.058003,0.941997,0.026384,0.015997,0.015997,0.084387,...,None,None,NaN,0,None,None,0.0,0.0,0.0,0.0,2020.0,None,None,46.0,1,Touchdown,2922.0,46800000000000,70095000000000,U.S. Bank Stadium,N/A (Indoors) Temp: Humidity: Wind:,10160000-0581-7529-f18a-5d17e1c95da1,8.0,0,PAT2,0,None,-9223372036854775808,None,15.0,Touchdown,None,4.0,4560000000000,3.0,0.0,1.0,3.0,4.0,0.0,KICKOFF,TOUCHDOWN,540000000000,49980000000000,MIN 23,GB 37,2790.0,2922.0,43.0,34.0,Home,-9.0,77.0,1.0,45.0,1,dome,sportturf,NaN,NaN,Mike Zimmer,Matt LaFleur,MIN01,U.S. Bank Stadium,1.0,None,NaN,D.Cook,33.0,None,NaN,0.0,1.0,0.0,0,0,1,None,00-0033893,None,D.C

In [199]:
print([col for col in df.columns if 'two' in col])

['two_point_conv_result', 'two_point_conversion_prob', 'two_point_attempt', 'defensive_two_point_attempt', 'defensive_two_point_conv']


In [200]:
df[['desc','two_point_attempt', 'two_point_conv_result',
    'passer_id', 'rusher_id' , 'receiver_id', 
   'passer_player_name', 'rusher_player_name', 'receiver_player_name',
   'passer_position', 'rusher_position', 'receiver_position']]

,desc,two_point_attempt,two_point_conv_result,passer_id,rusher_id,receiver_id,passer_player_name,rusher_player_name,receiver_player_name,passer_position,rusher_position,receiver_position
0,(Pass formation) TWO-POINT CONVERSION ATTEMPT. 33-D.Cook rushes up the middle. ATTEMPT SUCCEEDS.,1.0,success,None,00-0033893,None,None,D.Cook,None,None,RB,None
1,TWO-POINT CONVERSION ATTEMPT. 33-D.Cook rushes up the middle. ATTEMPT SUCCEEDS.,1.0,success,None,00-0033893,None,None,D.Cook,None,None,RB,None
2,"TWO-POINT CONVERSION ATTEMPT. 8-K.Cousins pass to 19-A.Thielen is complete. ATTEMPT SUCCEEDS. PENALTY on GB-20-K.King, Defensive Holding, 5 yards, enforced between downs.",1.0,success,00-0029604,None,00-0030035,K.Cousins,None,A.Thielen,QB,None,WR
3,(Pass formation) TWO-POINT CONVERSION ATTEMPT. 4-D.Watson pass to 18-R.Cobb is incomplete. ATTEMPT FAILS.,1.0,failure,00-0033537,None,00-0028002,D.Watson,None,R.Cobb,QB,None,WR
4,TWO-POINT CONVERSION ATTEMPT. 5-T.Bridgewater pass to 11-R.Anderson is complete. ATTEMPT SUCCEEDS.,1.0,success,00-0031237,None,00-0032688,T.Bridgewater,None,R.Anderson,QB,None,WR
...,...,...,...,...,...,...,...,...,...,...,...,...
138,(Kick formation) TWO-POINT CONVERSION ATTEMPT. 6-J.Scott pass to 2-M.Crosby is complete. ATTEMPT FAILS.,1.0,failure,00-0034162,None,00-0025580,J.Scott,None,M.Crosby,P,None,K
139,TWO-POINT CONVERSION ATTEMPT. 12-A.Rodgers pass to 33-A.Jones is incomplete. ATTEMPT FAILS.,1.0,failure,00-0023459,None,00-0033293,A.Rodgers,None,A.Jones,QB,None,RB
140,TWO-POINT CONVERSION ATTEMPT. 16-J.Goff pass to 23-C.Akers is complete. ATTEMPT SUCCEEDS. Lateral from 12-V.Jefferson to 23-C.Akers.,1.0,success,00-0033106,None,00-0036414,J.Goff,None,C.Akers,QB,None,RB
141,TWO-POINT CONVERSION ATTEMPT. 17-J.Allen pass to 14-S.Diggs is incomplete. ATTEMPT FAILS. DEFENSIVE TWO-POINT ATTEMPT. 21-B.Breeland intercepted the try attempt. ATTEMPT FAILS.,1.0,failure,00-0034857,None,00-0031588,J.Allen,None,S.Diggs,QB,None,WR


In [220]:
def _extract_qb_stats(db_conn) -> pd.DataFrame:
    """Getting passing stats, per player per game"""
    logging.info("Extracting receving stats by player by game from play by play...")
    query = """
        SELECT 
            year, season_type, game_id, posteam AS team, defteam AS opp, week,
            passer_id AS gsis_id, passer_position AS pos, passer_player_name AS player,
            COUNT(*) AS two_point_conversions
        FROM
            play_by_play_enriched
        WHERE
            two_point_attempt = 1 and two_point_conv_result = 'success'
            AND passer_id IS NOT NULL and passer_position IS NOT NULL
        GROUP BY
            year, season_type, game_id, posteam, defteam, week, passer_id, passer_position, passer_player_name
    """
    return pd.read_sql(query, db_conn)


def _extract_skill_position_stats(db_conn) -> pd.DataFrame:
    """Getting passing stats, per player per game"""
    logging.info("Extracting receving stats by player by game from play by play...")
    query = """
        SELECT 
            year, season_type, game_id, posteam AS team, defteam AS opp, week, fantasy_player_id AS gsis_id,
            CASE WHEN receiver_position IS NULL THEN rusher_position ELSE receiver_position END AS pos,
            fantasy_player_name AS player,
            COUNT(*) AS two_point_conversions
        FROM
            play_by_play_enriched
        WHERE
            two_point_attempt = 1 and two_point_conv_result = 'success'
            AND passer_id IS NOT NULL and passer_position IS NOT NULL
        GROUP BY
            year, season_type, game_id, posteam, defteam, week, fantasy_player_id, fantasy_player_name,
            (CASE WHEN receiver_position IS NULL THEN rusher_position ELSE receiver_position END)
    """
    return pd.read_sql(query, db_conn)


def _transform(df_qb, df_skill):
    return pd.concat([df_qb, df_skill])


df_qb = _extract_qb_stats(db_conn)
df_skill = _extract_skill_position_stats(db_conn)
df = _transform(df_qb, df_skill)
df.head(20)

,year,season_type,game_id,team,opp,week,gsis_id,pos,player,two_point_conversions
0,2020.0,POST,2020_18_CLE_PIT,PIT,CLE,18.0,00-0022924,QB,B.Roethlisberger,1
1,2020.0,POST,2020_18_IND_BUF,IND,BUF,18.0,00-0022942,QB,P.Rivers,1
2,2020.0,POST,2020_19_LA_GB,LA,GB,19.0,00-0033106,QB,J.Goff,1
3,2020.0,REG,2020_01_GB_MIN,MIN,GB,1.0,00-0029604,QB,K.Cousins,1
4,2020.0,REG,2020_01_LV_CAR,CAR,LV,1.0,00-0031237,QB,T.Bridgewater,1
5,2020.0,REG,2020_02_BUF_MIA,MIA,BUF,2.0,00-0023682,QB,R.Fitzpatrick,1
6,2020.0,REG,2020_02_DEN_PIT,DEN,PIT,2.0,00-0032436,QB,J.Driskel,1
7,2020.0,REG,2020_02_KC_LAC,KC,LAC,2.0,00-0033873,QB,P.Mahomes,1
8,2020.0,REG,2020_03_DAL_SEA,SEA,DAL,3.0,00-0029263,QB,R.Wilson,1
9,2020.0,REG,2020_03_HOU_PIT,PIT,HOU,3.0,00-0022924,QB,B.Roethlisberger,1


In [224]:
df_year = df.groupby(['year', 'season_type', 'team', 'gsis_id', 'pos', 'player'], as_index=False).sum()
df_year.drop('week', axis=1).sort_values('two_point_conversions', ascending=False).head(20)

,year,season_type,team,gsis_id,pos,player,two_point_conversions
48,2020.0,REG,MIN,00-0029604,QB,K.Cousins,5
62,2020.0,REG,PHI,00-0032950,QB,C.Wentz,5
34,2020.0,REG,KC,00-0033873,QB,P.Mahomes,3
22,2020.0,REG,DET,00-0026498,QB,M.Stafford,2
41,2020.0,REG,LAC,00-0036355,QB,J.Herbert,2
44,2020.0,REG,MIA,00-0023682,QB,R.Fitzpatrick,2
45,2020.0,REG,MIA,00-0035350,WR,P.Williams,2
52,2020.0,REG,MIN,00-0036322,WR,J.Jefferson,2
67,2020.0,REG,PIT,00-0022924,QB,B.Roethlisberger,2
50,2020.0,REG,MIN,00-0034970,TE,I.Smith,1


In [225]:
df.to_csv('2pt.csv', index=False)